# TP3 - Grupo 20
###  Dezembro, 4, 2024

Afonso Martins Campos Fernandes - A102940

Luís Filipe Pinheiro Silva - A105530

## Cifra A5/1 - SFOTS

Considere de novo o 1º problema do trabalho TP2  relativo à descrição da cifra $$\,\mathsf{A5/1}$$ e o FOTS usando BitVec’s que aì foi definido para a componente do gerador de chaves. Ignore a componente de geração final da chave e restrinja o modelo aos três LFSR’s. 
Sejam $$\,\mathsf{X}_0, \mathsf{X}_1, \mathsf{X}_2\,$$ as variáveis que determinam os estados dos três LFSR’s que ocorrem neste modelo. Como condição inicial  e condição de erro use os predicados

$$\,\mathsf{I} \;\equiv\; (\mathsf{X}_0 > 0)\,\land\,(\mathsf{X}_1 > 0)\,\land\,(\mathsf{X}_2 > 0)\quad e \quad \mathsf{E}\;\equiv\;\neg\,\mathsf{I}$$

In [28]:
from pysmt.shortcuts import *
from z3 import *

a. Codifique em “z3”  o SFOTS assim definido.

In [29]:
def states(i):
    state = {}

    state['LFSR1'] = BitVec('LFSR1' + str(i),19)
    state['LFSR2'] = BitVec('LFSR2' + str(i),22)
    state['LFSR3'] = BitVec('LFSR3' + str(i),23)

    return state

In [30]:
def init(state):
    return And(state['LFSR1'] > 0,state['LFSR2'] > 0,state['LFSR3'] > 0)

In [31]:
def error(state):
    return Not(And(state['LFSR1'] > 0,state['LFSR2'] > 0,state['LFSR3'] > 0))

In [32]:
def transition(atual,prox):
    cBit1 = Extract(8, 8, atual['LFSR1'])
    cBit2 = Extract(10, 10, atual['LFSR2'])
    cBit3 = Extract(10, 10, atual['LFSR3'])

    t1y = And(prox['LFSR1'] == Concat(Extract(18, 18, atual['LFSR1']) ^ 
                                      Extract(17, 17, atual['LFSR1']) ^ 
                                      Extract(16, 16, atual['LFSR1']) ^ 
                                      Extract(13, 13, atual['LFSR1']), 
                                      Extract(18, 1, atual['LFSR1'])),
              Or(cBit1 == cBit2,cBit1 == cBit3))
    
    t1n = And(prox['LFSR1'] == atual['LFSR1'], 
              Not(Or(cBit1 == cBit2,cBit1 == cBit3)))

    t2y = And(prox['LFSR2'] == Concat(Extract(21, 21, atual['LFSR2']) ^ 
                                      Extract(20, 20, atual['LFSR2']), 
                                      Extract(21, 1, atual['LFSR2'])),
              Or(cBit1 == cBit2,cBit2 == cBit3))
    
    t2n = And(prox['LFSR2'] == atual['LFSR2'], 
              Not(Or(cBit1 == cBit2,cBit2 == cBit3)))

    t3y = And(prox['LFSR3'] == Concat(Extract(22, 22, atual['LFSR3']) ^ 
                                      Extract(21, 21, atual['LFSR3']) ^ 
                                      Extract(20, 20, atual['LFSR3']) ^ 
                                      Extract(7, 7, atual['LFSR3']), 
                                      Extract(22, 1, atual['LFSR3'])),
              Or(cBit3 == cBit2,cBit1 == cBit3))
    
    t3n = And(prox['LFSR3'] == atual['LFSR3'], 
              Not(Or(cBit3 == cBit2,cBit1 == cBit3)))

    return And(Or(t1y,t1n),Or(t2y,t2n),Or(t3y,t3n))

In [33]:
def gera_sfots(init,error,trans,k):

    solver = Solver()

    estados = [states(i) for i in range(k)]

    solver.add(init(estados[0]))

    for i in range(k):
        solver.add(Not(error(estados[i])))

    for i in range(k-1):
        solver.add(trans(estados[i],estados[i+1]))

    check = solver.check()
    
    if check == sat:
        print("Is solvable")
        m = solver.model()
        for i in range(k):
            print("Passo",i+1)
            for v in estados[i]:
                x = m.eval(estados[i][v])
                print(v,"=", format(x.as_long(),f'0{m[estados[i][v]].size()}b'))
            print("--------------------------------")
    else:
        print("Not solvable")

gera_sfots(init,error,transition,10)

Is solvable
Passo 1
LFSR1 = 0000001111000100100
LFSR2 = 0000100111101010011001
LFSR3 = 01111001111010100010010
--------------------------------
Passo 2
LFSR1 = 0000000111100010010
LFSR2 = 0000010011110101001100
LFSR3 = 01111001111010100010010
--------------------------------
Passo 3
LFSR1 = 0000000011110001001
LFSR2 = 0000001001111010100110
LFSR3 = 00111100111101010001001
--------------------------------
Passo 4
LFSR1 = 0000000001111000100
LFSR2 = 0000000100111101010011
LFSR3 = 00111100111101010001001
--------------------------------
Passo 5
LFSR1 = 0000000000111100010
LFSR2 = 0000000010011110101001
LFSR3 = 00111100111101010001001
--------------------------------
Passo 6
LFSR1 = 0000000000011110001
LFSR2 = 0000000001001111010100
LFSR3 = 00111100111101010001001
--------------------------------
Passo 7
LFSR1 = 0000000000001111000
LFSR2 = 0000000000100111101010
LFSR3 = 00011110011110101000100
--------------------------------
Passo 8
LFSR1 = 0000000000000111100
LFSR2 = 00000000000100111101

b. Use o algoritmo PDR “property directed reachability” (codifique-o ou use uma versão pré-existente) e, com ele, tente provar a segurança deste modelo.

In [34]:
def states_PDR():
    LFSR1 = BitVec('LFSR1',19)
    LFSR2 = BitVec('LFSR2',22)
    LFSR3 = BitVec('LFSR3',23)

    return [LFSR1,LFSR2,LFSR3]

def next_states_PDR():
    n_LFSR1 = BitVec('n_LFSR1',19)
    n_LFSR2 = BitVec('n_LFSR2',22)
    n_LFSR3 = BitVec('n_LFSR3',23)

    return [n_LFSR1,n_LFSR2,n_LFSR3]

def init_PDR(s):

    return And(s[0] > 0, s[1] > 0, s[2] > 0)

def prop(s):

    return init_PDR(s)

def transition_PDR(atual,prox):
    cBit1 = Extract(8, 8, atual[0])
    cBit2 = Extract(10, 10, atual[1])
    cBit3 = Extract(10, 10, atual[2])

    t1y = And(prox[0] == Concat(Extract(18, 18, atual[0]) ^ 
                                      Extract(17, 17, atual[0]) ^ 
                                      Extract(16, 16, atual[0]) ^ 
                                      Extract(13, 13, atual[0]), 
                                      Extract(18, 1, atual[0])),
              Or(cBit1 == cBit2,cBit1 == cBit3))
    
    t1n = And(prox[0] == atual[0], 
              Not(Or(cBit1 == cBit2,cBit1 == cBit3)))

    t2y = And(prox[1] == Concat(Extract(21, 21, atual[1]) ^ 
                                      Extract(20, 20, atual[1]), 
                                      Extract(21, 1, atual[1])),
              Or(cBit1 == cBit2,cBit2 == cBit3))
    
    t2n = And(prox[1] == atual[1], 
              Not(Or(cBit1 == cBit2,cBit2 == cBit3)))

    t3y = And(prox[2] == Concat(Extract(22, 22, atual[2]) ^ 
                                      Extract(21, 21, atual[2]) ^ 
                                      Extract(20, 20, atual[2]) ^ 
                                      Extract(7, 7, atual[2]), 
                                      Extract(22, 1, atual[2])),
              Or(cBit3 == cBit2,cBit1 == cBit3))
    
    t3n = And(prox[2] == atual[2], 
              Not(Or(cBit3 == cBit2,cBit1 == cBit3)))

    return And(Or(t1y,t1n),Or(t2y,t2n),Or(t3y,t3n))

In [ ]:
# Nesta função não descrevi o P porque o estado de erro é a negação do init, logo como o P é a negação do estado de erro, P é igual ao init
def myPDR(trans):
    
    solver = Solver()
    frames = [Not(init_PDR(states_PDR()))]
    k = 0

    while True:
        print("Iteração",k)
        bad = get_bad_cubes(prop,frames,k,solver)

        if bad == None:
            if k > 0 and frames[k] == frames[k-1]:
                print("Sistema é seguro")
                return None
            else:
                frames.append(False)
                k+=1
        else:
            block = bloqueio(bad,frames,k,trans,solver)
            if not block:
                print("Sistema é inseguro")
                return None


def get_bad_cubes(prop,frames,k,solver):

    estado_atual = states_PDR()
    solver.push()
    solver.add(Not(frames[k]))
    solver.add(Not(prop(estado_atual)))

    if solver.check() == sat:
        model = solver.model()
        cube = And(estado_atual[0] == model.eval(estado_atual[0], model_completion=True),estado_atual[1] == model.eval(estado_atual[1], model_completion=True),estado_atual[2] == model.eval(estado_atual[2], model_completion=True))
        solver.pop()
        return cube
    
    solver.pop()
    return None

def bloqueio(bad_cube,frames,k,trans,solver):

    for i in range(k,0,-1):
        solver.push()
        estado_atual = states_PDR()
        proximo_estado = next_states_PDR()
        sub = list(zip(estado_atual,proximo_estado))

        solver.add(trans(estado_atual,proximo_estado))
        solver.add(Not(frames[i-1]))
        cubep = substitute(bad_cube,sub)
        solver.add(cubep)
        solver.add(Not(bad_cube))
        
        if solver.check() == unsat:
            solver.pop()
            print("Cubo bloqueado no frame",i)
            for j in range(0,i+1):
                if frames[j] != init:
                    frames[j] = And(frames[j],bad_cube)
                else:
                    frames[j] = bad_cube
            return True
        solver.pop()
    print(frames[k])
    return False

myPDR(transition_PDR)

Iteração 0
Iteração 1
Cubo bloqueado no frame 1


Z3Exception: Symbolic expressions cannot be cast to concrete Boolean values.

## Implementação com um algoritmo PDR pré-definido

In [ ]:
from pysmt.shortcuts import *
from pysmt.shortcuts import Symbol
from pysmt.typing import BVType
from z3 import And,Concat,Extract,Or,BitVec,Not
from pdr import PDR

x = BitVec('LFSR1',19)
y = BitVec('LFSR2',22)
z = BitVec('LFSR3',23)

xp = BitVec('LFSR1\'',19)
yp = BitVec('LFSR2\'',22)
zp = BitVec('LFSR3\'',23)

variables = [x,y,z]
primes = [xp,yp,zp]

init1 = And(x > 0, y > 0, z > 0)

cBit1 = Extract(8, 8, x)
cBit2 = Extract(10, 10, y)
cBit3 = Extract(10, 10, z)

t1y = And(xp == Concat(Extract(18, 18, x) ^ 
                       Extract(17, 17, x) ^ 
                       Extract(16, 16, x) ^ 
                       Extract(13, 13, x), 
                       Extract(18, 1, x)),
                Or(cBit1 == cBit2,cBit1 == cBit3))
    
t1n = And(xp == x, Not(Or(cBit1 == cBit2,cBit1 == cBit3)))

t2y = And(yp == Concat(Extract(21, 21, y) ^ 
                       Extract(20, 20, y), 
                       Extract(21, 1, y)),
                Or(cBit1 == cBit2,cBit2 == cBit3))
    
t2n = And(yp == y, Not(Or(cBit1 == cBit2,cBit2 == cBit3)))

t3y = And(zp == Concat(Extract(22, 22, z) ^ 
                       Extract(21, 21, z) ^ 
                       Extract(20, 20, z) ^ 
                       Extract(7, 7, z), 
                       Extract(22, 1, z)),
                Or(cBit3 == cBit2,cBit1 == cBit3))
    
t3n = And(zp == z, Not(Or(cBit3 == cBit2,cBit1 == cBit3)))



trans = And(Or(t1y,t1n),Or(t2y,t2n),Or(t3y,t3n))

post = init1

solver = PDR(variables,primes,init1,trans,post)
solver.run()

Did not find invariant, adding frame
Found trace ending in bad state:
1: [0 == LFSR2, 1 == LFSR1, 1 == LFSR3]
0: [1 == LFSR2, 2 == LFSR1, 2 == LFSR3]


'Sistema inseguro'